# Custom Prediction Routines on Google AI Platform

# Imports

In [ ]:
%matplotlib inline

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import base64

from googleapiclient import discovery
from PIL import Image

print('TensorFlow version:', tf.__version__)

# Testing the Pretrained Model

In [ ]:
classes = ['Cat', 'Dog']

model = tf.keras.models.load_model('model/cats_vs_dogs.h5')
model.summary()

In [ ]:
def display_prediction(image_path):
    
    plt.imshow(plt.imread(image_path))
    
    x = tf.keras.preprocessing.image.load_img(image_path, target_size=(128, 128, 3))
    x = tf.keras.preprocessing.image.img_to_array(x)
    x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
    x = np.expand_dims(x, axis=0)
    
    pred = model.predict(x)[0]
    
    plt.title('Prediction: {}'.format(classes[int(pred > 0.5)]))
    plt.show()
    
    print(np.squeeze(pred))

In [ ]:
display_prediction('images/dog.jpg')

In [ ]:
display_prediction('images/cat.jpg')

In [ ]:
display_prediction('images/dog2.jpg')

In [ ]:
display_prediction('images/cat2.jpg')

# Creating Custom Prediction Class

In [ ]:
%%writefile prediction.py
import tensorflow as tf
import numpy as np
import os
import base64

MODEL_NAME = 'cats_vs_dogs.h5'
DEFAULT_SIZE = 128
CLASS_NAMES = ['Cat', 'Dog']

class CatsVsDogsPrediction:
    def __init__(self, model):
        self._model = model
    
    def _preprocess(self, instances):
        x_batch = np.zeros((len(instances), self.size, self.size, 3))
        for i, instance in enumerate(instances):
            x = np.asarray(bytearray(base64.b64decode(instance)))
            x = np.reshape(x, (self.size, self.size, 3))
            x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
            x_batch[i] = x
        return x_batch
    
    def _postprocess(self, predictions):
        results = []
        for i, prediction in enumerate(predictions):
            results.append(
                {
                    'index': i,
                    'class_name': CLASS_NAMES[int(np.squeeze(prediction) > 0.5)],
                    'raw_value': '{:.4f}'.format(float(np.squeeze(prediction)))
                }
            )
        return results
    
    def predict(self, instances, **kwargs):
        if 'size' in kwargs:
            self.size = int(kwargs.get('size'))
        else:
            self.size = DEFAULT_SIZE
        x_batch = self._preprocess(instances)
        preds = self._model.predict(x_batch)
        results = self._postprocess(preds)
        return results

    @classmethod
    def from_path(cls, model_dir):
        model = tf.keras.models.load_model(os.path.join(model_dir, MODEL_NAME))
        return cls(model)

# Custom Prediction Package

In [ ]:
%%writefile setup.py
from setuptools import setup

setup(
     name="cats_vs_dogs",
     version="0.0.1",
     include_package_data=False,
     scripts=["prediction.py"]
)

In [ ]:
!python3 setup.py sdist --formats=gztar

# Using the Deployed Model

In [ ]:
service = discovery.build('ml', 'v1', cache_discovery=False)

def get_prediction_models_api(body, project_name, model_name):
    response = service.projects().predict(
        name = 'projects/{}/models/{}'.format(project_name, model_name),
        body = body
    ).execute()
    return response

In [ ]:
project_name = 'YOUR_PROJECT_NAME'
model_name = 'YOUR_MODEL_NAME_ON_AI_PLATFORM'
instances = []
size = 128

images = ['dog.jpg', 'cat.jpg', 'dog2.jpg', 'cat2.jpg']

for image in images:
    img = Image.open(os.path.join('images', image))
    img = img.resize((size, size), Image.ANTIALIAS)
    
    instances.append(base64.b64encode(img.tobytes()).decode())
    
    img.close()

In [ ]:
body = {
    'instances': instances,
    'size': size
}

response = get_prediction_models_api(body, project_name, model_name)

In [ ]:
response